# Recon2.2plusthreeCompetition.json [Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [4]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

0.16888753778858653

# Recon2.2plus with exercise (Other treatment)

In [8]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [9]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

In [10]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [11]:
model=M.copy()

In [12]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [13]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [14]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [15]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.PHPYRte.upper_bound=0
model1.reactions.PHPYRte.lower_bound=0
model1.reactions.EX_phpyr_e_INB.upper_bound=0
sol = model1.optimize()
sol.objective_value

0.18734518083433543

In [16]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [18]:
#Remembering to add other amino acid
model1.reactions.EX_glc_e.lower_bound=-2##Do not need much glucsoe.
model1.reactions.EX_his_L_e.lower_bound=-0.2
model1.reactions.EX_ile_L_e.lower_bound=-0.2
model1.reactions.EX_leu_L_e.lower_bound=-0.2
model1.reactions.EX_lys_L_e.lower_bound=-0.2
model1.reactions.EX_met_L_e.lower_bound=-0.2
model1.reactions.EX_thr_L_e.lower_bound=-0.2
model1.reactions.EX_trp_L_e.lower_bound=-0.1
model1.reactions.EX_val_L_e.lower_bound=-0.2

In [19]:
model1.reactions.biomass_reaction.upper_bound=0.5
sol = model1.optimize()
sol.objective_value

0.24190296608223896

In [20]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# Young PKU patient VS elderly PKU patient with Recon2.2plusthreeCompetitive

## Young PKU patients

In [43]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [44]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

In [45]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [46]:
model=M.copy()

In [47]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [48]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [49]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [50]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.PHPYRte.upper_bound=0
model1.reactions.PHPYRte.lower_bound=0
model1.reactions.EX_phpyr_e_INB.upper_bound=0
sol = model1.optimize()
sol.objective_value

0.18734518083433543

In [51]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [52]:
# For children
model2=model1.copy()
model2.solver='glpk'

In [53]:
model2.reactions.biomass_reaction_INB.upper_bound=0.1
model2.reactions.biomass_reaction_INB.lower_bound=0.1
model2.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.0596
model2.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.0596
model2.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model2.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model2.reactions.EX_phe_L_e.upper_bound=-0.039
model2.reactions.EX_phe_L_e.lower_bound=-0.039
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##For children, the basic need for Phe is 0.0389. And if they need the biomass, dopamine and serotonin in the brain be normal, they should decrease the Phe uptake flux to 0.04.

In [54]:
#For adult person
model3=model1.copy()
model3.reactions.biomass_reaction_INB.upper_bound=0.05
model3.reactions.biomass_reaction_INB.lower_bound=0.05
model3.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.0596
model3.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.0596
model3.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model3.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model3.reactions.EX_phe_L_e.upper_bound=-0.0475
model3.reactions.EX_phe_L_e.lower_bound=-0.0475
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [55]:
#For old person
model4=model1.copy()
model4.reactions.biomass_reaction_INB.upper_bound=0.02
model4.reactions.biomass_reaction_INB.lower_bound=0.02
model4.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.07
model4.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.07
model4.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model4.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model4.reactions.EX_phe_L_e.upper_bound=-0.0425
model4.reactions.EX_phe_L_e.lower_bound=-0.0425
fbasol = model4.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.004787, '1a_24…

In [63]:
#For old person (less biomass and more dopamine)
model5=model1.copy()
model5.reactions.biomass_reaction_INB.upper_bound=0.02
model5.reactions.biomass_reaction_INB.lower_bound=0.02
model5.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.06
model5.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.06
model5.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model5.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model5.reactions.EX_phe_L_e.upper_bound=-0.0524
model5.reactions.EX_phe_L_e.lower_bound=-0.0524
fbasol = model5.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# Parkinson analyze with Recon2.2plusthreeCompetitive

## Parkinson only

In [64]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [65]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

In [66]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [67]:
model=M.copy()

In [68]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [69]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.1
model.reactions.EX_trp_L_e.lower_bound=-0.12

In [70]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [71]:
model1=model.copy()
# Simulating the Parkinson disease
model1.reactions.get_by_id('TYRDOPO1_INB').upper_bound=0.02

In [72]:
sol = model1.optimize()
sol.objective_value

0.23800409999999997

In [73]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [74]:
## Providing extra L-dopa
model2=model1.copy()
model2.reactions.EX_tyr_L_e.upper_bound=-0.1
model2.reactions.get_by_id('EX_Ldopa_BD').lower_bound=-0.04

In [75]:
sol = model2.optimize()
sol.objective_value

0.2547554000000001

In [76]:
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [77]:
model2.reactions.EX_trp_L_e.lower_bound=-0.1
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##In this case, the PD patients should also control their diet, otherwise the treated effection from levedopa will not be that good.

## Parkinson with PKU disease together

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

/Users/yanhualiu/Desktop/New_topic/model_building/model_building_exercise_by_yanhua/for different models/Competition between Phe, Tyr, Trp and L-dopa/Making brain specific model and complex model/utils/findBiomarkers.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [4]:
model=M.copy()

In [5]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [6]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.1
model.reactions.EX_phe_L_e.upper_bound=-0.1
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [7]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [14]:
model1=model.copy()
# Simulating the Parkinson disease
model1.reactions.TYRDOPO1_INB.upper_bound=0.02
##PKU+Parkinson disease
model1.genes.get_by_id('HGNC:8582').knock_out()
# No phenylpyruvate production in the liver
model1.reactions.PHPYRte.upper_bound=0
model1.reactions.PHPYRte.lower_bound=0
model1.reactions.EX_phpyr_e_INB.upper_bound=0

In [15]:
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value

0.18734518083433804

In [16]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [19]:
### Adding levodopa in the medium
model1.reactions.get_by_id('EX_Ldopa_BD').lower_bound=-0.05
model1.reactions.get_by_id('EX_Ldopa_BD').upper_bound=-0.05
model1.reactions.get_by_id('EX_34dhphe_e').upper_bound=1

In [20]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [21]:
model1.reactions.EX_phe_L_e.upper_bound=-0.05
model1.reactions.EX_phe_L_e.lowerer_bound=-0.05
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# Biomarker finding with Recon2.2plusthreeCompetitive

In [38]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [39]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitivemap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

In [31]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_co2_e.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0

In [32]:
model.reactions.EX_dopa_e_INB.upper_bound=1000
model.reactions.EX_adrnl_e_INB.upper_bound=1000
model.reactions.EX_srtn_e_INB.upper_bound=1000
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# No phenylpyruvate exchange in the brain
model.reactions.get_by_id('EX_phpyr_e_INB').upper_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.LdopaTR_TOBD.upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti_To.upper_bound=0.198

In [33]:
import cobra
from utils import findBiomarkers
import pandas as pd

In [34]:
exchanges = [ rxn for rxn in model.reactions if rxn.products == [] and 'EX_' in rxn.id and 'INB' not in rxn.id]

# Shlomi et al suggested using the following medium: everything in (-1) with a few exceptions 
# everything out (unlimited)
# This was actually first proposed in (Sahoo et al, 2012)
for rxn in exchanges:
    rxn.lower_bound = -1
    rxn.upper_bound = 999999
        
# specifics
model.reactions.EX_o2_e.lower_bound = -40
model.reactions.r0399.lower_bound=0

for rxn in ['EX_h2o_e','EX_h_e','EX_co2_e','EX_nh4_e','EX_pi_e','EX_hco3_e','EX_so4_e']:
    model.reactions.get_by_id(rxn).lower_bound = - 100

# to reduce computation time we check all amino acids + a couple neurotransmitters
biomarkers_to_check = ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e',
                       'EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e','EX_cys_L_e',
                       'EX_glu_L_e','EX_tyr_L_e','EX_ala_L_e','EX_asp_L_e','EX_gly_e',
                       'EX_arg_L_e','EX_gln_L_e','EX_pro_L_e','EX_ser_L_e','EX_asn_L_e',
                        'EX_dopa_e','EX_adrnl_e','EX_srtn_e']

# UNCOMMENT & FIX THE LINE BELOW
# threshold higher than 0.1 will show.
model.solver = 'glpk'
findBiomarkers(model,fvaRxns=exchanges,mods=['HGNC:8582'],synchronous=True,fracOpt=0.5)

Interpreting mode as IEMgene
Finding reactions associated with modifier genes by knockout...
Modifications will be performed on the following reactions:
                                                       Description                                                                        Reaction
PHETHPTOX2_INB  L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c_INB + phe_L_c_INB + thbpt_c_INB --> thbpt4acam_c_INB + tyr_L_c_INB       
PHETHPTOX2      L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c + phe_L_c + thbpt_c --> thbpt4acam_c + tyr_L_c                           
r0399_INB       L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c_INB + phe_L_c_INB + thbpt_c_INB --> dhbpt_c_INB + h2o_c_INB + tyr_L_c_INB
r0399           L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c + phe_L_c + thbpt_c --> dhbpt_c + h2o_c + tyr_L_c                        

190 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,
34hpp_e,3-(4-hydroxyphenyl)pyruvate,Reduced,"[-1.0, 3.979]","[-1.0, 0.992]",0.750691
tyr_L_e,L-tyrosine,Reduced,"[-1.0, 3.979]","[-1.0, 0.992]",0.750691
tymsf_e,Tyramine O-sulfate,Reduced,"[0.0, 4.979]","[0.0, 1.992]",0.599920
4hphac_e,4-hydroxyphenylacetate,Reduced,"[0.0, 4.979]","[0.0, 1.992]",0.599920
q10_e,coenzyme Q10,Reduced,"[-1.0, 5.979]","[-1.0, 2.992]",0.499582
q10h2_e,ubiquinol-10,Reduced,"[-1.0, 5.979]","[-1.0, 2.992]",0.499582
34dhphe_e,L-dopa,Reduced,"[-1.0, 5.979]","[-1.0, 2.992]",0.499582
dopa_e,dopaminium(1+),Reduced,"[-1.0, 6.979]","[-1.0, 3.992]",0.427998
dopasf_e,dopamine 3-O-sulfate,Reduced,"[0.0, 7.979]","[0.0, 4.992]",0.374358


In [35]:
exchanges = [ rxn for rxn in model.reactions if rxn.products == [] and 'EX_' in rxn.id and 'INB' not in rxn.id]

# Shlomi et al suggested using the following medium: everything in (-1) with a few exceptions 
# everything out (unlimited)
# This was actually first proposed in (Sahoo et al, 2012)
for rxn in exchanges:
    rxn.lower_bound = -1
    rxn.upper_bound = 999999
        
# specifics
model.reactions.EX_o2_e.lower_bound = -40
model.reactions.r0399.lower_bound=0

for rxn in ['EX_h2o_e','EX_h_e','EX_co2_e','EX_nh4_e','EX_pi_e','EX_hco3_e','EX_so4_e']:
    model.reactions.get_by_id(rxn).lower_bound = - 100

# to reduce computation time we check all amino acids + a couple neurotransmitters
biomarkers_to_check = ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e',
                       'EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e','EX_cys_L_e',
                       'EX_glu_L_e','EX_tyr_L_e','EX_ala_L_e','EX_asp_L_e','EX_gly_e',
                       'EX_arg_L_e','EX_gln_L_e','EX_pro_L_e','EX_ser_L_e','EX_asn_L_e',
                        'EX_dopa_e','EX_adrnl_e','EX_srtn_e']

# UNCOMMENT & FIX THE LINE BELOW
# threshold higher than 0.1 will show.
model.solver = 'glpk'
findBiomarkers(model,fvaRxns=exchanges,mods=['HGNC:8582'],synchronous=True,fracOpt=1)

Interpreting mode as IEMgene
Finding reactions associated with modifier genes by knockout...
Modifications will be performed on the following reactions:
                                                       Description                                                                        Reaction
PHETHPTOX2_INB  L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c_INB + phe_L_c_INB + thbpt_c_INB --> thbpt4acam_c_INB + tyr_L_c_INB       
PHETHPTOX2      L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c + phe_L_c + thbpt_c --> thbpt4acam_c + tyr_L_c                           
r0399_INB       L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c_INB + phe_L_c_INB + thbpt_c_INB --> dhbpt_c_INB + h2o_c_INB + tyr_L_c_INB
r0399           L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c + phe_L_c + thbpt_c --> dhbpt_c + h2o_c + tyr_L_c                        

190 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,
34hpp_e,3-(4-hydroxyphenyl)pyruvate,Reduced,"[-1.0, 3.937]","[-1.0, 0.976]",0.752096
tyr_L_e,L-tyrosine,Reduced,"[-1.0, 3.937]","[-1.0, 0.976]",0.752096
tymsf_e,Tyramine O-sulfate,Reduced,"[0.0, 4.937]","[0.0, 1.976]",0.599757
4hphac_e,4-hydroxyphenylacetate,Reduced,"[0.0, 4.937]","[0.0, 1.976]",0.599757
q10_e,coenzyme Q10,Reduced,"[-1.0, 5.937]","[-1.0, 2.976]",0.498737
q10h2_e,ubiquinol-10,Reduced,"[-1.0, 5.937]","[-1.0, 2.976]",0.498737
34dhphe_e,L-dopa,Reduced,"[-1.0, 5.937]","[-1.0, 2.976]",0.498737
dopa_e,dopaminium(1+),Reduced,"[-1.0, 6.937]","[-1.0, 3.976]",0.426842
dopasf_e,dopamine 3-O-sulfate,Reduced,"[0.0, 7.937]","[0.0, 4.976]",0.373063


In [37]:
exchanges = [ rxn for rxn in model.reactions if rxn.products == [] and 'EX_' in rxn.id and 'INB' not in rxn.id]

# Shlomi et al suggested using the following medium: everything in (-1) with a few exceptions 
# everything out (unlimited)
# This was actually first proposed in (Sahoo et al, 2012)
for rxn in exchanges:
    rxn.lower_bound = -1
    rxn.upper_bound = 999999
        
# specifics
model.reactions.EX_o2_e.lower_bound = -40
model.reactions.r0399.lower_bound=0

for rxn in ['EX_h2o_e','EX_h_e','EX_co2_e','EX_nh4_e','EX_pi_e','EX_hco3_e','EX_so4_e']:
    model.reactions.get_by_id(rxn).lower_bound = - 100

# to reduce computation time we check all amino acids + a couple neurotransmitters
biomarkers_to_check = ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e',
                       'EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e','EX_cys_L_e',
                       'EX_glu_L_e','EX_tyr_L_e','EX_ala_L_e','EX_asp_L_e','EX_gly_e',
                       'EX_arg_L_e','EX_gln_L_e','EX_pro_L_e','EX_ser_L_e','EX_asn_L_e',
                        'EX_dopa_e','EX_adrnl_e','EX_srtn_e']

# UNCOMMENT & FIX THE LINE BELOW
# threshold higher than 0.1 will show.
model.solver = 'glpk'
findBiomarkers(model,fvaRxns=exchanges,mods=['HGNC:8582'],synchronous=True,fracOpt=0)

Interpreting mode as IEMgene
Finding reactions associated with modifier genes by knockout...
Modifications will be performed on the following reactions:
                                                       Description                                                                        Reaction
PHETHPTOX2_INB  L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c_INB + phe_L_c_INB + thbpt_c_INB --> thbpt4acam_c_INB + tyr_L_c_INB       
PHETHPTOX2      L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c + phe_L_c + thbpt_c --> thbpt4acam_c + tyr_L_c                           
r0399_INB       L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c_INB + phe_L_c_INB + thbpt_c_INB --> dhbpt_c_INB + h2o_c_INB + tyr_L_c_INB
r0399           L-Phenylalanine,tetrahydrobiopterin:oxygen oxidore  o2_c + phe_L_c + thbpt_c --> dhbpt_c + h2o_c + tyr_L_c                        

190 low confidence biomarkers with scores below the threshold were                 found


,Name,Prediction,WT,Mutant,Score
ID,,,,,
34hpp_e,3-(4-hydroxyphenyl)pyruvate,Reduced,"[-1.0, 3.979]","[-1.0, 0.992]",0.750691
tyr_L_e,L-tyrosine,Reduced,"[-1.0, 3.979]","[-1.0, 0.992]",0.750691
tymsf_e,Tyramine O-sulfate,Reduced,"[-0.0, 4.979]","[0.0, 1.992]",0.599920
4hphac_e,4-hydroxyphenylacetate,Reduced,"[0.0, 4.979]","[0.0, 1.992]",0.599920
q10_e,coenzyme Q10,Reduced,"[-1.0, 5.979]","[-1.0, 2.992]",0.499582
q10h2_e,ubiquinol-10,Reduced,"[-1.0, 5.979]","[-1.0, 2.992]",0.499582
34dhphe_e,L-dopa,Reduced,"[-1.0, 5.979]","[-1.0, 2.992]",0.499582
dopa_e,dopaminium(1+),Reduced,"[-1.0, 6.979]","[-1.0, 3.992]",0.427998
dopasf_e,dopamine 3-O-sulfate,Reduced,"[0.0, 7.979]","[0.0, 4.992]",0.374358
